<a href="https://colab.research.google.com/github/Meenusiwach/CSSBATTLE_2024/blob/main/twitteranalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
!mkdir .p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘.p’: File exists
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
!kaggle datasets download -d kazanova/sentiment140

 77% 62.0M/80.9M [00:00<00:00, 217MB/s]
100% 80.9M/80.9M [00:00<00:00, 191MB/s]


In [ ]:
#extracrting the zip file
from zipfile import ZipFile
dataset ='/content/sentiment140.zip'

with ZipFile(dataset, 'r') as zip:
   zip.extractall()
   print('the dataset is extracted')

the dataset is extracted


In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding ='ISO-8859-1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
column_names=['target','id','date','flag','user','text']
twitter_data =pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names,encoding='ISO-8859-1')



In [ ]:
twitter_data.shape

(1600000, 6)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#countig a number of missing values in dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

In [ ]:
port_stem=PorterStemmer()


In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]', ' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content =' '.join(stemmed_content)

  return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

In [ ]:
print(twitter_data['stememd_content'])

In [ ]:
print(twitter_data'[target]')

In [ ]:
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

In [ ]:
print(Y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size =0.2 ,stratify=Y,random_state=2)

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

In [ ]:
print(X_test)

In [ ]:
vectorizer = TfidfVectorizer()
X_train =vectorizer.fit_transform(X_train)
X_test=vectorizer.transform(X_test)


In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
model=LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
X_train_prediction=model.predict(X_train)
training_data_accuracy - accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('accuracy is :',trainig_data_accuracy)

In [ ]:
X_test_prediction=model.predict(X_test)
testing_data_accuracy - accuracy_score(Y_test,X_test_prediction)

In [ ]:
print('accuracy is :',testing_data_accuracy)

In [ ]:
import pickle

In [ ]:
filename='trained_model.sav'
pickle.dump(model,open(filename,'wb'))


In [ ]:
loaded_model =pickle.load(open('','rb'))

In [ ]:
X_new =X_test[200]
print(Y_test[200])

prediction =model.predict(X_new)
print(prediction)
if(prediction[0]==0)
   print('Negative Tweet')
else:
    print('positive Tweet')

In [ ]:
X_new =X_test[3]
print(Y_test[3])

prediction =model.predict(X_new)
print(prediction)
if(prediction[0]==0)
   print('Negative Tweet')
else:
    print('positive Tweet')